In [1]:
import json
import os

# ✅ Kaggle API 설정 (보안 문제 해결)
api_dict = {
    "username": "dogwalwal",
    "key": "5245358191529d040a3411ab9fb97bec"
}

# kaggle.json 파일 생성
kaggle_path = "/root/.kaggle"
os.makedirs(kaggle_path, exist_ok=True)

with open(os.path.join(kaggle_path, "kaggle.json"), "w") as f:
    json.dump(api_dict, f)

# 권한 설정
os.chmod(os.path.join(kaggle_path, "kaggle.json"), 600)


In [2]:
# ✅ Titanic 데이터 다운로드
!kaggle competitions download -c titanic -p /content

# ✅ 압축 해제
import zipfile

with zipfile.ZipFile("/content/titanic.zip", "r") as zip_ref:
    zip_ref.extractall("/content")


titanic.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression
import warnings

warnings.filterwarnings("ignore")  # ⚠️ 불필요한 경고 메시지 숨기기

# ✅ 원본 데이터 로드
df_original = pd.read_csv('/content/train.csv')

datasets = {}  # 생성된 데이터셋 저장 딕셔너리

# ✅ Age 결측치 처리 함수
def process_age(df, method):
    df = df.copy()
    if method == "mean":
        df["Age"].fillna(df["Age"].mean(), inplace=True)
    elif method == "median":
        df["Age"].fillna(df["Age"].median(), inplace=True)
    elif method == "knn":
        imputer = KNNImputer(n_neighbors=5)
        df["Age"] = imputer.fit_transform(df[["Age"]])
    elif method == "group_median":
        df["Age"] = df.groupby(["Pclass", "Sex"])["Age"].transform(lambda x: x.fillna(x.median()))
    elif method == "regression":
        age_df = df.dropna(subset=["Age"])
        X_train = age_df.drop(columns=["Age", "Cabin", "Embarked", "Name", "Ticket", "Sex"])
        X_train = pd.get_dummies(X_train, columns=["Pclass"])
        y_train = age_df["Age"]
        model = LinearRegression()
        model.fit(X_train, y_train)

        X_test = df.loc[df["Age"].isnull(), :].drop(columns=["Age", "Cabin", "Embarked", "Name", "Ticket", "Sex"])
        X_test = pd.get_dummies(X_test, columns=["Pclass"])
        df.loc[df["Age"].isnull(), "Age"] = model.predict(X_test)
    return df

# ✅ Cabin 결측치 처리 함수
def process_cabin(df, method):
    df = df.copy()
    if method == "unknown":
        df["Cabin"].fillna("Unknown", inplace=True)
    elif method == "deck":
        df["Deck"] = df["Cabin"].str[0]  # ✅ 첫 글자 (선실 구역) 추출
        df["Deck"].fillna("U", inplace=True)  # ✅ "U"로 채움
        df.drop(columns=["Cabin"], inplace=True)  # ✅ Cabin 제거
    elif method == "has_cabin":
        df["Has_Cabin"] = df["Cabin"].notnull().astype(int)  # ✅ Cabin 여부 저장
        df.drop(columns=["Cabin"], inplace=True)
    return df

# ✅ Embarked 결측치 처리 함수
def process_embarked(df, method):
    df = df.copy()
    if method == "mode":
        df["Embarked"].fillna(df["Embarked"].mode()[0], inplace=True)
    elif method == "group_fare":
        df["Embarked"] = df.groupby(["Pclass", "Fare"])["Embarked"].transform(
            lambda x: x.fillna(x.mode()[0] if not x.mode().empty else "S")
        )
    return df

# ✅ 모든 조합에 대해 데이터 생성
age_methods = ["mean", "median", "knn", "group_median", "regression"]
cabin_methods = ["unknown", "deck", "has_cabin"]
embarked_methods = ["mode", "group_fare"]

for a in age_methods:
    for c in cabin_methods:
        for e in embarked_methods:
            df_processed = df_original.copy()
            df_processed = process_age(df_processed, a)
            df_processed = process_cabin(df_processed, c)
            df_processed = process_embarked(df_processed, e)
            datasets[f"A{age_methods.index(a)+1}_C{cabin_methods.index(c)+1}_E{embarked_methods.index(e)+1}"] = df_processed

# ✅ 저장된 데이터셋 개수 및 샘플 확인
print(f"총 {len(datasets)}개의 데이터셋이 생성되었습니다.")
for key, df in list(datasets.items())[:3]:  # 일부 샘플 확인
    print(f"\n▶ {key} 데이터셋 샘플:")
    print(df.head())


총 30개의 데이터셋이 생성되었습니다.

▶ A1_C1_E1 데이터셋 샘플:
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare    Cabin Embarked  
0      0         A/5 21171   7.2500  Unknown        S  
1      0          PC 17599  71.2833      C85        C  
2      0  STON/O2. 3101282   7.9250  Unknown        S  
3      0            113803  53.1000     C123        S

In [4]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html


In [10]:
import pandas as pd
import h2o
from h2o.automl import H2OAutoML
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
import warnings

warnings.filterwarnings("ignore")  # ⚠️ 불필요한 경고 메시지 숨기기

# ✅ H2O 클러스터 초기화
h2o.init()

# ✅ 원본 데이터 로드
df_original = pd.read_csv('/content/train.csv')

datasets = {}  # 생성된 데이터셋 저장 딕셔너리

# ✅ Age 결측치 처리 함수
def process_age(df, method):
    df = df.copy()
    if method == "mean":
        df["Age"].fillna(df["Age"].mean(), inplace=True)
    elif method == "median":
        df["Age"].fillna(df["Age"].median(), inplace=True)
    elif method == "knn":
        imputer = KNNImputer(n_neighbors=5)
        df["Age"] = imputer.fit_transform(df[["Age"]])
    elif method == "group_median":
        df["Age"] = df.groupby(["Pclass", "Sex"])["Age"].transform(lambda x: x.fillna(x.median()))
    elif method == "regression":
        age_df = df.dropna(subset=["Age"])
        X_train = age_df.drop(columns=["Age", "Cabin", "Embarked", "Name", "Ticket", "Sex"])
        X_train = pd.get_dummies(X_train, columns=["Pclass"])
        y_train = age_df["Age"]
        model = LinearRegression()
        model.fit(X_train, y_train)

        X_test = df.loc[df["Age"].isnull(), :].drop(columns=["Age", "Cabin", "Embarked", "Name", "Ticket", "Sex"])
        X_test = pd.get_dummies(X_test, columns=["Pclass"])
        df.loc[df["Age"].isnull(), "Age"] = model.predict(X_test)
    return df

# ✅ Cabin 결측치 처리 함수
def process_cabin(df, method):
    df = df.copy()
    if method == "unknown":
        df["Cabin"].fillna("Unknown", inplace=True)
    elif method == "deck":
        df["Deck"] = df["Cabin"].str[0]
        df["Deck"].fillna("U", inplace=True)
        df.drop(columns=["Cabin"], inplace=True)
    elif method == "has_cabin":
        df["Has_Cabin"] = df["Cabin"].notnull().astype(int)
        df.drop(columns=["Cabin"], inplace=True)
    return df

# ✅ Embarked 결측치 처리 함수
def process_embarked(df, method):
    df = df.copy()
    if method == "mode":
        df["Embarked"].fillna(df["Embarked"].mode()[0], inplace=True)
    elif method == "group_fare":
        df["Embarked"] = df.groupby(["Pclass", "Fare"])["Embarked"].transform(
            lambda x: x.fillna(x.mode()[0] if not x.mode().empty else "S")
        )
    return df

# ✅ 모든 조합에 대해 데이터 생성
age_methods = ["mean", "median", "knn", "group_median", "regression"]
cabin_methods = ["unknown", "deck", "has_cabin"]
embarked_methods = ["mode", "group_fare"]

target = "Survived"
for a in age_methods:
    for c in cabin_methods:
        for e in embarked_methods:
            df_processed = df_original.copy()
            df_processed = process_age(df_processed, a)
            df_processed = process_cabin(df_processed, c)
            df_processed = process_embarked(df_processed, e)

            # ✅ H2O 경고 방지: 불필요한 컬럼 제거 & 범주형 변환
            df_processed = df_processed.drop(columns=["Name"])  # Name 컬럼 미리 삭제
            df_processed["Survived"] = df_processed["Survived"].astype("category")  # pandas 단계에서 범주형 변환

            datasets[f"A{age_methods.index(a)+1}_C{cabin_methods.index(c)+1}_E{embarked_methods.index(e)+1}"] = df_processed

# ✅ H2O AutoML 실행 및 평가
results = {}
leaderboard_models = {}

for key, df in datasets.items():
    print(f"\n▶ {key} 데이터셋 AutoML 실행 중...")

    # ✅ H2O 프레임 변환
    h2o_df = h2o.H2OFrame(df)

    # ✅ H2O 경고 방지: 타겟을 명확하게 범주형(factor)으로 설정
    h2o_df["Survived"] = h2o_df["Survived"].asfactor()

    train, test = h2o_df.split_frame(ratios=[0.8], seed=42)

    # ✅ AutoML 실행
    aml = H2OAutoML(max_models=10, seed=42)
    aml.train(y=target, training_frame=train)

    # ✅ 최적 모델 선택 및 예측
    best_model = aml.leader
    pred = best_model.predict(test).as_data_frame()["predict"].round().astype(int)  # 확률 → 0 또는 1 변환
    true = test[target].as_data_frame().astype(int)  # 정수형 변환

    # ✅ 정확도 계산
    accuracy = accuracy_score(true, pred)
    results[key] = accuracy

    # ✅ 리더보드 저장
    leaderboard_models[key] = aml.leaderboard.as_data_frame()

# ✅ 결과 정렬 및 출력
sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)
print("\n✅ 최적의 데이터셋 조합:")
for key, acc in sorted_results[:5]:  # 상위 5개 출력
    print(f"{key}: {acc:.4f}")

# ✅ 상위 조합에서 사용된 모델 출력
print("\n✅ 상위 조합에서 사용된 모델:")
for key, acc in sorted_results[:3]:  # 상위 3개 데이터셋의 모델 정보 확인
    print(f"\n▶ {key} 데이터셋 리더보드 모델:")

    # 리더보드 컬럼 확인
    leaderboard = leaderboard_models[key]
    available_metrics = leaderboard.columns  # 실제 리더보드에 있는 컬럼

    # 출력할 주요 컬럼 선택 (우선순위: auc > logloss > rmse > mse)
    selected_metrics = ["auc", "logloss", "rmse", "mse"]
    valid_metrics = [m for m in selected_metrics if m in available_metrics]  # 실제 존재하는 컬럼만 선택

    # 존재하는 메트릭만 출력
    print(leaderboard[["model_id"] + valid_metrics])

# ✅ H2O 클러스터 종료
h2o.shutdown(prompt=False)


Checking whether there is an H2O instance running at http://localhost:54321. connected.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,3 mins 10 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,3 months and 13 days
H2O_cluster_name:,H2O_from_python_unknownUser_ufkp8n
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.157 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"



▶ A1_C1_E1 데이터셋 AutoML 실행 중...

▶ A1_C1_E2 데이터셋 AutoML 실행 중...

▶ A1_C2_E1 데이터셋 AutoML 실행 중...

▶ A1_C2_E2 데이터셋 AutoML 실행 중...

▶ A1_C3_E1 데이터셋 AutoML 실행 중...

▶ A1_C3_E2 데이터셋 AutoML 실행 중...

▶ A2_C1_E1 데이터셋 AutoML 실행 중...

▶ A2_C1_E2 데이터셋 AutoML 실행 중...

▶ A2_C2_E1 데이터셋 AutoML 실행 중...

▶ A2_C2_E2 데이터셋 AutoML 실행 중...

▶ A2_C3_E1 데이터셋 AutoML 실행 중...

▶ A2_C3_E2 데이터셋 AutoML 실행 중...

▶ A3_C1_E1 데이터셋 AutoML 실행 중...

▶ A3_C1_E2 데이터셋 AutoML 실행 중...

▶ A3_C2_E1 데이터셋 AutoML 실행 중...

▶ A3_C2_E2 데이터셋 AutoML 실행 중...

▶ A3_C3_E1 데이터셋 AutoML 실행 중...

▶ A3_C3_E2 데이터셋 AutoML 실행 중...

▶ A4_C1_E1 데이터셋 AutoML 실행 중...

▶ A4_C1_E2 데이터셋 AutoML 실행 중...

▶ A4_C2_E1 데이터셋 AutoML 실행 중...

▶ A4_C2_E2 데이터셋 AutoML 실행 중...

▶ A4_C3_E1 데이터셋 AutoML 실행 중...

▶ A4_C3_E2 데이터셋 AutoML 실행 중...

▶ A5_C1_E1 데이터셋 AutoML 실행 중...

▶ A5_C1_E2 데이터셋 AutoML 실행 중...

▶ A5_C2_E1 데이터셋 AutoML 실행 중...

▶ A5_C2_E2 데이터셋 AutoML 실행 중...

▶ A5_C3_E1 데이터셋 AutoML 실행 중...

▶ A5_C3_E2 데이터셋 AutoML 실행 중...

✅ 최적의 데이터셋 조합:
A5_C3_E1: 0.8663
A5_C3_E